In [8]:
import numpy as np

def objective(x):
    a, b, c = x
    return (a - 0.5)**2 + (b - 15)**2 + (c - 1.5)**2  # Example function

# Bounds for each variable
bounds = np.array([[0, 1], [10, 20], [0, 3]])

# Initial guess
x0 = np.array([0.9, 19, 0.1])

def mads(func, x0, bounds, delta=1.0, tol=1e-6, max_iter=100):
    """ Mesh Adaptive Direct Search (MADS) implementation. """
    n = len(x0)
    x = x0.copy()
    delta_min = tol
    iteration = 0
    
    while delta > delta_min and iteration < max_iter:
        improved = False
        
        # Generate trial points
        for i in range(n):
            for direction in [-1, 1]:
                x_new = x.copy()
                x_new[i] += direction * delta
                x_new = np.clip(x_new, bounds[:, 0], bounds[:, 1])  # Enforce bounds
                
                if func(x_new) < func(x):
                    x = x_new
                    improved = True
        
        # Reduce mesh size if no improvement
        if not improved:
            delta /= 2.0
        
        iteration += 1
        yield x, func(x), iteration

# Run MADS optimization
mads_iter = mads(objective, x0, bounds)
for x, fval, step in mads_iter:
    print(f"Step {step}: x = {x}, f(x) = {fval}")

Step 1: x = [ 0.9 18.   1.1], f(x) = 9.32
Step 2: x = [ 0.9 17.   1.1], f(x) = 4.32
Step 3: x = [ 0.9 16.   1.1], f(x) = 1.32
Step 4: x = [ 0.9 15.   1.1], f(x) = 0.31999999999999995
Step 5: x = [ 0.9 15.   1.1], f(x) = 0.31999999999999995
Step 6: x = [ 0.4 15.   1.6], f(x) = 0.02000000000000001
Step 7: x = [ 0.4 15.   1.6], f(x) = 0.02000000000000001
Step 8: x = [ 0.4 15.   1.6], f(x) = 0.02000000000000001
Step 9: x = [ 0.525 15.     1.475], f(x) = 0.0012499999999999968
Step 10: x = [ 0.525 15.     1.475], f(x) = 0.0012499999999999968
Step 11: x = [ 0.525 15.     1.475], f(x) = 0.0012499999999999968
Step 12: x = [ 0.49375 15.       1.50625], f(x) = 7.812500000000083e-05
Step 13: x = [ 0.49375 15.       1.50625], f(x) = 7.812500000000083e-05
Step 14: x = [ 0.49375 15.       1.50625], f(x) = 7.812500000000083e-05
Step 15: x = [ 0.5015625 15.         1.4984375], f(x) = 4.882812499999793e-06
Step 16: x = [ 0.5015625 15.         1.4984375], f(x) = 4.882812499999793e-06
Step 17: x = [ 0.501

In [15]:
import numpy as np

class MADS:
    def __init__(self, x0, bounds, delta=1.0, tol=1e-6, max_iter=100):
        self.x = np.array(x0)
        self.bounds = np.array(bounds)
        self.delta = delta
        self.delta_min = tol
        self.max_iter = max_iter
        self.iteration = 0
        self.history = [(self.x.copy(), float('inf'))]  # Store initial point
        self.waiting_for_reward = False
        self.last_trial = None
    
    def step(self, reward=None):
        """ Perform one step of MADS. Provide reward when available. """
        if self.waiting_for_reward:
            if reward is None:
                raise ValueError("Reward must be provided for the last trial step.")
            
            # Accept the trial point if the reward is better
            if reward < self.history[-1][1]:
                self.x = self.last_trial
            self.history.append((self.x.copy(), reward))
            self.waiting_for_reward = False
            
        if self.delta <= self.delta_min or self.iteration >= self.max_iter:
            return None  # Termination condition
        
        improved = False
        trial_points = []
        
        for i in range(len(self.x)):
            for direction in [-1, 1]:
                x_new = self.x.copy()
                x_new[i] += direction * self.delta
                x_new = np.clip(x_new, self.bounds[:, 0], self.bounds[:, 1])
                trial_points.append(x_new)
        
        self.last_trial = trial_points.pop(0)  # Pick the first trial point
        self.waiting_for_reward = True
        self.iteration += 1
        return self.last_trial




In [8]:
# Example usage:
bounds = [(0, 1), (10, 20), (0, 3)]
x0 = [0.5, 12, 1]
mads = MADS(x0, bounds)



In [9]:
x_next = mads.step()

In [10]:
reward = (x_next[0] - 0.5)**2 + (x_next[1] - 15)**2 + (x_next[2] - 1.5)**2

In [11]:
reward

np.float64(9.5)

In [12]:
mads.step(reward)

array([ 0., 12.,  1.])

In [14]:
mads.history

[(array([ 0.5, 12. ,  1. ]), inf), (array([ 0., 12.,  1.]), np.float64(9.5))]

In [16]:
while True:
    x_next = mads.step()
    if x_next is None:
        break
    
    # Simulate delayed function evaluation
    reward = (x_next[0] - 0.5)**2 + (x_next[1] - 15)**2 + (x_next[2] - 1.5)**2
    print("x, reward", x_next, reward)
    mads.step(reward)

ValueError: Reward must be provided for the last trial step.

In [20]:
import numpy as np

class MADS:
    def __init__(self, x0, bounds, delta=1.0, tol=1e-6, max_iter=100):
        self.x = np.array(x0)
        self.bounds = np.array(bounds)
        self.delta = delta
        self.delta_min = tol
        self.max_iter = max_iter
        self.iteration = 0
        self.history = []
        self.waiting_for_reward = False
        self.last_trial = None
    
    def get_next_trial(self):
        """ Generate the next trial point for evaluation. """
        if self.waiting_for_reward:
            raise RuntimeError("Provide a reward for the last trial before requesting a new one.")
        
        if self.delta <= self.delta_min or self.iteration >= self.max_iter:
            return None  # Termination condition
        
        trial_points = []
        for i in range(len(self.x)):
            for direction in [-1, 1]:
                x_new = self.x.copy()
                x_new[i] += direction * self.delta
                x_new = np.clip(x_new, self.bounds[:, 0], self.bounds[:, 1])
                trial_points.append(x_new)
        
        self.last_trial = trial_points.pop(0)  # Pick the first trial point
        self.waiting_for_reward = True
        return self.last_trial
    
    def update_with_reward(self, reward):
        """ Update the MADS algorithm with the reward from the last trial. """
        if not self.waiting_for_reward:
            raise RuntimeError("No trial is pending a reward update.")
        
        if not self.history or reward < self.history[-1][1]:
            self.x = self.last_trial
        
        self.history.append((self.x.copy(), reward))
        self.waiting_for_reward = False
        self.iteration += 1
        
        if not any(reward < r for _, r in self.history):
            self.delta /= 2.0  # Reduce the mesh size if no improvement

# Example usage:
bounds = [(0, 1), (10, 20), (0, 3)]
x0 = [0.5, 12, 1]
mads = MADS(x0, bounds)

while True:
    x_next = mads.get_next_trial()
    print("x_next",x_next)
    if x_next is None:
        break
    
    # Simulate delayed function evaluation
    reward = (x_next[0] - 0.5)**2 + (x_next[1] - 15)**2 + (x_next[2] - 1.5)**2
    print(reward)
    mads.update_with_reward(reward)


x_next [ 0. 12.  1.]
9.5
x_next [ 0. 12.  1.]
9.5
x_next [ 0. 12.  1.]
9.5
x_next [ 0. 12.  1.]
9.5
x_next [ 0. 12.  1.]
9.5
x_next [ 0. 12.  1.]
9.5
x_next [ 0. 12.  1.]
9.5
x_next [ 0. 12.  1.]
9.5
x_next [ 0. 12.  1.]
9.5
x_next [ 0. 12.  1.]
9.5
x_next [ 0. 12.  1.]
9.5
x_next [ 0. 12.  1.]
9.5
x_next [ 0. 12.  1.]
9.5
x_next [ 0. 12.  1.]
9.5
x_next [ 0. 12.  1.]
9.5
x_next [ 0. 12.  1.]
9.5
x_next [ 0. 12.  1.]
9.5
x_next [ 0. 12.  1.]
9.5
x_next [ 0. 12.  1.]
9.5
x_next [ 0. 12.  1.]
9.5
x_next None


In [23]:
import numpy as np

class MADS:
    def __init__(self, x0, bounds, delta=1.0, tol=1e-6, max_iter=100):
        self.x = np.array(x0)
        self.bounds = np.array(bounds)
        self.delta = delta
        self.delta_min = tol
        self.max_iter = max_iter
        self.iteration = 0
        self.history = []
        self.waiting_for_reward = False
        self.last_trial = None
        self.trial_queue = []
    
    def get_next_trial(self):
        """ Generate the next trial point for evaluation. """
        if self.waiting_for_reward:
            raise RuntimeError("Provide a reward for the last trial before requesting a new one.")
        
        if self.delta <= self.delta_min or self.iteration >= self.max_iter:
            return None  # Termination condition
        
        if not self.trial_queue:
            # Generate new trial points only when the queue is empty
            for i in range(len(self.x)):
                for direction in [-1, 1]:
                    x_new = self.x.copy()
                    x_new[i] += direction * self.delta
                    x_new = np.clip(x_new, self.bounds[:, 0], self.bounds[:, 1])
                    self.trial_queue.append(x_new)
        
        self.last_trial = self.trial_queue.pop(0)  # Pick the next trial point
        self.waiting_for_reward = True
        return self.last_trial
    
    def update_with_reward(self, reward):
        """ Update the MADS algorithm with the reward from the last trial. """
        if not self.waiting_for_reward:
            raise RuntimeError("No trial is pending a reward update.")
        
        self.history.append((self.last_trial.copy(), reward))
        if not self.history or reward < min(r for _, r in self.history):
            self.x = self.last_trial  # Accept the trial if it improves
        
        self.waiting_for_reward = False
        self.iteration += 1
        
        if not self.trial_queue and all(reward >= r for _, r in self.history[-len(self.x)*2:]):
            self.delta /= 2.0  # Reduce the mesh size if no improvement

# Example usage:
bounds = [(0, 1), (10, 20), (0, 3)]
x0 = [0.5, 12, 1]
mads = MADS(x0, bounds)

while True:
    x_next = mads.get_next_trial()
    if x_next is None:
        break
    
    # Simulate delayed function evaluation
    reward = (x_next[0] - 0.5)**2 + (x_next[1] - 15)**2 + (x_next[2] - 1.5)**2
    print("x_next", x_next, reward)
    mads.update_with_reward(reward)

x_next [ 0. 12.  1.] 9.5
x_next [ 1. 12.  1.] 9.5
x_next [ 0.5 11.   1. ] 16.25
x_next [ 0.5 13.   1. ] 4.25
x_next [ 0.5 12.   0. ] 11.25
x_next [ 0.5 12.   2. ] 9.25
x_next [ 0. 12.  1.] 9.5
x_next [ 1. 12.  1.] 9.5
x_next [ 0.5 11.   1. ] 16.25
x_next [ 0.5 13.   1. ] 4.25
x_next [ 0.5 12.   0. ] 11.25
x_next [ 0.5 12.   2. ] 9.25
x_next [ 0. 12.  1.] 9.5
x_next [ 1. 12.  1.] 9.5
x_next [ 0.5 11.   1. ] 16.25
x_next [ 0.5 13.   1. ] 4.25
x_next [ 0.5 12.   0. ] 11.25
x_next [ 0.5 12.   2. ] 9.25
x_next [ 0. 12.  1.] 9.5
x_next [ 1. 12.  1.] 9.5
x_next [ 0.5 11.   1. ] 16.25
x_next [ 0.5 13.   1. ] 4.25
x_next [ 0.5 12.   0. ] 11.25
x_next [ 0.5 12.   2. ] 9.25
x_next [ 0. 12.  1.] 9.5
x_next [ 1. 12.  1.] 9.5
x_next [ 0.5 11.   1. ] 16.25
x_next [ 0.5 13.   1. ] 4.25
x_next [ 0.5 12.   0. ] 11.25
x_next [ 0.5 12.   2. ] 9.25
x_next [ 0. 12.  1.] 9.5
x_next [ 1. 12.  1.] 9.5
x_next [ 0.5 11.   1. ] 16.25
x_next [ 0.5 13.   1. ] 4.25
x_next [ 0.5 12.   0. ] 11.25
x_next [ 0.5 12.   2.

In [5]:
a = "True"

In [6]:
a.lower()=="true"

True